# Sentiment Analysis with an RNN

In this notebook, you'll implement a recurrent neural network that performs sentiment analysis. Using an RNN rather than a feedfoward network is more accurate since we can include information about the *sequence* of words. Here we'll use a dataset of movie reviews, accompanied by labels.

The architecture for this network is shown below.

<img src="assets/network_diagram.png" width=400px>

Here, we'll pass in words to an embedding layer. We need an embedding layer because we have tens of thousands of words, so we'll need a more efficient representation for our input data than one-hot encoded vectors. You should have seen this before from the word2vec lesson. You can actually train up an embedding with word2vec and use it here. But it's good enough to just have an embedding layer and let the network learn the embedding table on it's own.

From the embedding layer, the new representations will be passed to LSTM cells. These will add recurrent connections to the network so we can include information about the sequence of words in the data. Finally, the LSTM cells will go to a sigmoid output layer here. We're using the sigmoid because we're trying to predict if this text has positive or negative sentiment. The output layer will just be a single unit then, with a sigmoid activation function.

We don't care about the sigmoid outputs except for the very last one, we can ignore the rest. We'll calculate the cost from the output of the last step and the training label.

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
with open('./reviews.txt', 'r') as f:
    reviews = f.read()
with open('./labels.txt', 'r') as f:
    labels = f.read()

In [4]:
reviews[:2000]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   \nstory of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is tu

## Data preprocessing

The first step when building a neural network model is getting your data into the proper form to feed into the network. Since we're using embedding layers, we'll need to encode each word with an integer. We'll also want to clean it up a bit.

You can see an example of the reviews data above. We'll want to get rid of those periods. Also, you might notice that the reviews are delimited with newlines `\n`. To deal with those, I'm going to split the text into each review using `\n` as the delimiter. Then I can combined all the reviews back together into one big string.

First, let's remove all punctuation. Then get all the text without the newlines and split it into individual words.

In [15]:
reviews[0]

'b'

In [16]:
c = reviews[0]
c

'b'

In [17]:
str1 = ['b', 'o', 'y', 'g', 'i', 'r', 'l']
str1

['b', 'o', 'y', 'g', 'i', 'r', 'l']

> join element in list with nothing

In [19]:
text =''.join(str1)
text

'boygirl'

In [20]:
from string import punctuation
'\n' in punctuation

False

In [21]:
',' in punctuation

True

> c is character in below cell, 

> the second line join all characters except of punctuation

> Note '\n' is NOT in punctuation

> reviews and words  are what we want

In [5]:
from string import punctuation
all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')

all_text = ' '.join(reviews)
words = all_text.split()

In [31]:
reviews[0]

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers   the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students  when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled          at           high  a classic line inspector i  m here to sack one of your teachers  student welcome to bromwell high  i expect that many adults of my age think that bromwell high is far fetched  what a pity that it isn  t   '

In [35]:
all_text[0]

'b'

In [36]:
words[:100]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me',
 'to',
 'believe',
 'that',
 'bromwell',
 'high',
 's',
 'satire',
 'is',
 'much',
 'closer',
 'to',
 'reality',
 'than',
 'is',
 'teachers',
 'the',
 'scramble',
 'to',
 'survive',
 'financially',
 'the',
 'insightful',
 'students',
 'who',
 'can',
 'see',
 'right',
 'through',
 'their',
 'pathetic',
 'teachers',
 'pomp',
 'the',
 'pettiness',
 'of',
 'the',
 'whole',
 'situation',
 'all',
 'remind',
 'me',
 'of',
 'the',
 'schools',
 'i',
 'knew',
 'and',
 'their',
 'students',
 'when',
 'i',
 'saw',
 'the',
 'episode',
 'in',
 'which',
 'a',
 'student',
 'repeatedly',
 'tried',
 'to',
 'burn',
 'down',
 'the',
 'school',
 'i',
 'immediately',
 'recalled',
 'at',
 'high']

### Encoding the words

The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

> **Exercise:** Now you're going to encode the words with integers. Build a dictionary that maps words to integers. Later we're going to pad our input vectors with zeros, so make sure the integers **start at 1, not 0**.
> Also, convert the reviews to integers and store the reviews in a new list called `reviews_ints`. 

In [57]:
type(reviews[0])

str

In [55]:
reviews[0].split() == reviews[0].split(' ')

False

In [56]:
# Create your dictionary that maps vocab words to integers here
vocab = set(words)
vocab_to_int = {w: i+1 for i, w in enumerate(vocab)}

# Convert the reviews to integers, same shape as reviews list, but with integers
reviews_ints = list()
for i in range(len(reviews)):
    review = reviews[i].split()
    print('review:', review)
    print('\n')
    for ii in range(len(review)):
        print('review[ii]:', review[ii])
        print('\n')
        print(vocab_to_int[review[ii]])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


 bromwell


54014
review[ii]: high


34878
review[ii]: is


39646
review[ii]: a


33087
review[ii]: cartoon


10592
review[ii]: comedy


33884
review[ii]: it


70513
review[ii]: ran


66225
review[ii]: at


5640
review[ii]: the


22483
review[ii]: same


42950
review[ii]: time


24534
review[ii]: as


70129
review[ii]: some


52674
review[ii]: other


4308
review[ii]: programs


55851
review[ii]: about


57899
review[ii]: school


15290
review[ii]: life


35711
review[ii]: such


1413
review[ii]: as


70129
review[ii]: teachers


44323
review[ii]: my


25897
review[ii]: years


50632
review[ii]: in


40130
review[ii]: the


22483
review[ii]: teaching


23102
review[ii]: profession


20580
review[ii]: lead


4793
review[ii]: me


22025
review[ii]: to


7858
review[ii]: believe


63787
review[ii]: that


43433
review[ii]: bromwell


54014
review[ii]: high


34878
review[ii]: s


26937
review[ii]: satire


70644
review[ii]: is


39646
review[ii]: much


10024
review[ii]: closer


35028
re

42895
review[ii]: they


31066
review[ii]: re


55699
review[ii]: survivors


63889
review[ii]: bolt


70583
review[ii]: isn


57435
review[ii]: t


11339
review[ii]: he


61308
review[ii]: s


26937
review[ii]: not


66217
review[ii]: used


12317
review[ii]: to


7858
review[ii]: reaching


57408
review[ii]: mutual


28932
review[ii]: agreements


10431
review[ii]: like


61721
review[ii]: he


61308
review[ii]: once


3517
review[ii]: did


61640
review[ii]: when


72788
review[ii]: being


21687
review[ii]: rich


10679
review[ii]: where


30069
review[ii]: it


70513
review[ii]: s


26937
review[ii]: fight


29089
review[ii]: or


17790
review[ii]: flight


7505
review[ii]: kill


6394
review[ii]: or


17790
review[ii]: be


54471
review[ii]: killed


26763
review[ii]: br


57051
review[ii]: br


57051
review[ii]: while


45456
review[ii]: the


22483
review[ii]: love


64385
review[ii]: connection


72204
review[ii]: between


58723
review[ii]: molly


20367
review[ii]: and


469

review[ii]: missed


64480
review[ii]: opportunity


51222
review[ii]: while


45456
review[ii]: the


22483
review[ii]: rather


48654
review[ii]: sluggish


59
review[ii]: plot


7218
review[ii]: keeps


71273
review[ii]: one


9190
review[ii]: entertained


1450
review[ii]: for


32953
review[ii]: odd


48414
review[ii]: minutes


47234
review[ii]: not


66217
review[ii]: that


43433
review[ii]: much


10024
review[ii]: happens


24904
review[ii]: after


6788
review[ii]: the


22483
review[ii]: plane


50385
review[ii]: sinks


26140
review[ii]: there


6512
review[ii]: s


26937
review[ii]: not


66217
review[ii]: as


70129
review[ii]: much


10024
review[ii]: urgency


59756
review[ii]: as


70129
review[ii]: i


27254
review[ii]: thought


45884
review[ii]: there


6512
review[ii]: should


49971
review[ii]: have


51833
review[ii]: been


26330
review[ii]: even


45017
review[ii]: when


72788
review[ii]: the


22483
review[ii]: navy


67014
review[ii]: become


13761
review[

review[ii]: than


32782
review[ii]: usual


4921
review[ii]: the


22483
review[ii]: hospital


71946
review[ii]: scene


71500
review[ii]: and


46937
review[ii]: the


22483
review[ii]: scene


71500
review[ii]: where


30069
review[ii]: the


22483
review[ii]: homeless


64060
review[ii]: invade


28657
review[ii]: a


33087
review[ii]: demolition


25600
review[ii]: site


6570
review[ii]: are


20085
review[ii]: all


57595
review[ii]: time


24534
review[ii]: classics


16191
review[ii]: look


33778
review[ii]: for


32953
review[ii]: the


22483
review[ii]: legs


22043
review[ii]: scene


71500
review[ii]: and


46937
review[ii]: the


22483
review[ii]: two


64817
review[ii]: big


53558
review[ii]: diggers


51803
review[ii]: fighting


40465
review[ii]: one


9190
review[ii]: bleeds


36266
review[ii]: this


52403
review[ii]: movie


17829
review[ii]: gets


65794
review[ii]: better


17458
review[ii]: each


121
review[ii]: time


24534
review[ii]: i


27254
review[ii]: 

...

review[ii]: the


22483
review[ii]: guests


23611
review[ii]: he


61308
review[ii]: dies


19209
review[ii]: early


7296
review[ii]: in


40130
review[ii]: the


22483
review[ii]: film


27838
review[ii]: or


17790
review[ii]: does


71718
review[ii]: he


61308
review[ii]: and


46937
review[ii]: the


22483
review[ii]: residents


72434
review[ii]: of


63948
review[ii]: the


22483
review[ii]: house


4052
review[ii]: are


20085
review[ii]: subjected


16662
review[ii]: to


7858
review[ii]: a


33087
review[ii]: number


31353
review[ii]: of


63948
review[ii]: terrifying


48494
review[ii]: experiences


56628
review[ii]: i


27254
review[ii]: won


36562
review[ii]: t


11339
review[ii]: go


27847
review[ii]: into


7080
review[ii]: too


23430
review[ii]: much


10024
review[ii]: detail


11205
review[ii]: here


39807
review[ii]: but


21918
review[ii]: it


70513
review[ii]: is


39646
review[ii]: definitely


64821
review[ii]: a


33087
review[ii]: must


68684
review[i

12226
review[ii]: wrote


69048
review[ii]: dhoom


65711
review[ii]: and


46937
review[ii]: dhoom


65711
review[ii]: both


62473
review[ii]: of


63948
review[ii]: which


13587
review[ii]: i


27254
review[ii]: enjoyed


48756
review[ii]: tremendously


25023
review[ii]: br


57051
review[ii]: br


57051
review[ii]: in


40130
review[ii]: his


38592
review[ii]: rookie


67700
review[ii]: directorial


46159
review[ii]: outing


9552
review[ii]: with


51325
review[ii]: tashan


63837
review[ii]: while


45456
review[ii]: you


31774
review[ii]: can


10609
review[ii]: t


11339
review[ii]: fault


4231
review[ii]: his


38592
review[ii]: direction


4600
review[ii]: you


31774
review[ii]: d


14942
review[ii]: probably


15790
review[ii]: scratch


12970
review[ii]: your


72497
review[ii]: head


35211
review[ii]: over


4422
review[ii]: the


22483
review[ii]: plot


7218
review[ii]: which


13587
review[ii]: was


1316
review[ii]: clunky


35471
review[ii]: at


5640
review[i

review[ii]: able


19324
review[ii]: to


7858
review[ii]: despite


30761
review[ii]: the


22483
review[ii]: obvious


2279
review[ii]: language


6861
review[ii]: gaps


8363
review[ii]: emerge


1253
review[ii]: from


36714
review[ii]: screenings


54101
review[ii]: humming


59769
review[ii]: a


33087
review[ii]: tune


54372
review[ii]: or


17790
review[ii]: two


64817
review[ii]: i


27254
review[ii]: wasn


6927
review[ii]: t


11339
review[ii]: able


19324
review[ii]: to


7858
review[ii]: do


17501
review[ii]: that


43433
review[ii]: after


6788
review[ii]: tashan


63837
review[ii]: because


25852
review[ii]: the


22483
review[ii]: songs


29812
review[ii]: unfortunately


41451
review[ii]: just


18081
review[ii]: weren


18111
review[ii]: t


11339
review[ii]: catchy


71426
review[ii]: at


5640
review[ii]: all


57595
review[ii]: usually


50704
review[ii]: the


22483
review[ii]: song


61254
review[ii]: dance


59248
review[ii]: routine


68102
review[ii]: wo



57899
review[ii]: it


70513
review[ii]: without


45389
review[ii]: giving


6732
review[ii]: any


59299
review[ii]: kind


2813
review[ii]: of


63948
review[ii]: warning


32068
review[ii]: let


65237
review[ii]: s


26937
review[ii]: just


18081
review[ii]: say


13860
review[ii]: that


43433
review[ii]: this


52403
review[ii]: movie


17829
review[ii]: is


39646
review[ii]: like


61721
review[ii]: an


47250
review[ii]: exercise


69944
review[ii]: in


40130
review[ii]: cinema


47607
review[ii]: but


21918
review[ii]: really


62137
review[ii]: really


62137
review[ii]: great


24682
review[ii]: done


71568
review[ii]: its


54814
review[ii]: made


26819
review[ii]: with


51325
review[ii]: super


40831
review[ii]: black


26782
review[ii]: and


46937
review[ii]: white


49974
review[ii]: shots


47362
review[ii]: milimeters


63065
review[ii]: color


69752
review[ii]: interviews


31089
review[ii]: flashbacks


31146
review[ii]: aro


30027
review[ii]: tolbukhin

KeyboardInterrupt: 

## solution

In [6]:
from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

reviews_ints = []
for each in reviews:
    reviews_ints.append([vocab_to_int[word] for word in each.split()])

In [89]:
len(reviews_ints)

25001

### Encoding the labels

Our labels are "positive" or "negative". To use these labels in our network, we need to convert them to 0 and 1.

> **Exercise:** Convert labels from `positive` and `negative` to 1 and 0, respectively.

In [7]:
# Convert labels to 1s and 0s for 'positive' and 'negative'
labels = (np.array(labels.split('\n')) == 'positive').astype(int)

In [8]:
labels.shape

(25001,)

If you built `labels` correctly, you should see the next output.

In [9]:
from collections import Counter
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


Okay, a couple issues here. We seem to have one review with zero length. And, the maximum review length is way too many steps for our RNN. Let's truncate to 200 steps. For reviews shorter than 200, we'll pad with 0s. For reviews longer than 200, we can truncate them to the first 200 characters.

> **Exercise:** First, remove the review with zero length from the `reviews_ints` list.

In [93]:
# Filter out that review with 0 length
reviews_ints = [ review for review in reviews_ints if len(review) != 0]

> labels also needed to remove

# solution

In [31]:
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]
len(non_zero_idx)
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
labels = np.array([labels[ii] for ii in non_zero_idx])

In [33]:
labels.shape

(25000,)

In [11]:
from collections import Counter
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 0
Maximum review length: 2514


> **Exercise:** Now, create an array `features` that contains the data we'll pass to the network. The data should come from `review_ints`, since we want to feed integers to the network. Each row should be 200 elements long. For reviews shorter than 200 words, left pad with 0s. That is, if the review is `['best', 'movie', 'ever']`, `[117, 18, 128]` as integers, the row will look like `[0, 0, 0, ..., 0, 117, 18, 128]`. For reviews longer than 200, use on the first 200 words as the feature vector.

This isn't trivial and there are a bunch of ways to do this. But, if you're going to be building your own deep learning networks, you're going to have to get used to preparing your data.



In [12]:
reviews_ints[0]

[21025,
 308,
 6,
 3,
 1050,
 207,
 8,
 2138,
 32,
 1,
 171,
 57,
 15,
 49,
 81,
 5785,
 44,
 382,
 110,
 140,
 15,
 5194,
 60,
 154,
 9,
 1,
 4975,
 5852,
 475,
 71,
 5,
 260,
 12,
 21025,
 308,
 13,
 1978,
 6,
 74,
 2395,
 5,
 613,
 73,
 6,
 5194,
 1,
 24103,
 5,
 1983,
 10166,
 1,
 5786,
 1499,
 36,
 51,
 66,
 204,
 145,
 67,
 1199,
 5194,
 19869,
 1,
 37442,
 4,
 1,
 221,
 883,
 31,
 2988,
 71,
 4,
 1,
 5787,
 10,
 686,
 2,
 67,
 1499,
 54,
 10,
 216,
 1,
 383,
 9,
 62,
 3,
 1406,
 3686,
 783,
 5,
 3483,
 180,
 1,
 382,
 10,
 1212,
 13583,
 32,
 308,
 3,
 349,
 341,
 2913,
 10,
 143,
 127,
 5,
 7690,
 30,
 4,
 129,
 5194,
 1406,
 2326,
 5,
 21025,
 308,
 10,
 528,
 12,
 109,
 1448,
 4,
 60,
 543,
 102,
 12,
 21025,
 308,
 6,
 227,
 4146,
 48,
 3,
 2211,
 12,
 8,
 215,
 23]

In [34]:
seq_len = 200
features = list()
for review in reviews_ints:
    len_review = len(review)
    if len_review > 200:
        features.append(review[0:200])
    elif len_review == 200:
        features.append(review)
    else:
        temp = [0] * (seq_len -  len_review) + review 
        features.append(temp)
features = np.array(features)

If you build features correctly, it should look like that cell output below.

In [14]:
features[:10,:100]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 21025,   308,     6,
            3,  1050,   207,     8,  2138,    32,     1,   171,    57,
           15,    49,    81,  5785,    44,   382,   110,   140,    15,
         5194,    60,   154,     9,     1,  4975,  5852,   475,    71,
            5,   260,    12, 21025,   308,    13,  1978,     6,    74,
         2395],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     

In [35]:
features.shape

(25000, 200)

## Training, Validation, Test



With our data in nice shape, we'll split it into training, validation, and test sets.

> **Exercise:** Create the training, validation, and test sets here. You'll need to create sets for the features and the labels, `train_x` and `train_y` for example. Define a split fraction, `split_frac` as the fraction of data to keep in the training set. Usually this is set to 0.8 or 0.9. The rest of the data will be split in half to create the validation and testing data.

In [42]:
split_frac = 0.8
index_split = np.int(labels.shape[0] * split_frac)
print(index_split)
train_x, val_x = features[: index_split, :], features[index_split:, :]
train_y, val_y = labels[: index_split], labels[index_split:]

index_split2 = np.int(val_y.shape[0] * 0.5)
print(val_y.shape[0])
print(index_split2)
val_x, test_x = val_x[: index_split2, :], val_x[index_split2:, :]
val_y, test_y = val_y[: index_split2], val_y[index_split2:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

20000
5000
2500
			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


With train, validation, and text fractions of 0.8, 0.1, 0.1, the final shapes should look like:
```
                    Feature Shapes:
Train set: 		 (20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		  (2500, 200)
```

In [37]:
train_x

array([[    0,     0,     0, ...,     8,   215,    23],
       [    0,     0,     0, ...,    29,   108,  3324],
       [22382,    42, 46418, ...,   483,    17,     3],
       ..., 
       [    0,     0,     0, ...,    28,    77,   384],
       [    0,     0,     0, ...,     1,  1893,  3610],
       [    0,     0,     0, ...,     2,  2428,     8]])

In [38]:
train_x.shape

(20000, 200)

In [39]:
train_y.shape

(20000,)

> add one dimension to labels

In [43]:
train_y = train_y.reshape(-1, 1)

In [44]:
train_y.shape

(20000, 1)

In [45]:
val_y = val_y.reshape(-1, 1)
test_y = test_y.reshape(-1, 1)

In [46]:
val_y.shape

(2500, 1)

In [47]:
test_y.shape

(2500, 1)

In [48]:
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


In [49]:
print("\t\t\tlabes Shapes:")
print("Train set: \t\t{}".format(train_y.shape), 
      "\nValidation set: \t{}".format(val_y.shape),
      "\nTest set: \t\t{}".format(test_y.shape))

			labes Shapes:
Train set: 		(20000, 1) 
Validation set: 	(2500, 1) 
Test set: 		(2500, 1)


## Build the graph

Here, we'll build the graph. First up, defining the hyperparameters.

* `lstm_size`: Number of units in the hidden layers in the LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
* `lstm_layers`: Number of LSTM layers in the network. I'd start with 1, then add more if I'm underfitting.
* `batch_size`: The number of reviews to feed the network in one training pass. Typically this should be set as high as you can go without running out of memory.
* `learning_rate`: Learning rate

In [107]:
lstm_size = 256
lstm_layers = 1
batch_size = 500
learning_rate = 0.001

For the network itself, we'll be passing in our 200 element long review vectors. Each batch will be `batch_size` vectors. We'll also be using dropout on the LSTM layer, so we'll make a placeholder for the keep probability.

> **Exercise:** Create the `inputs_`, `labels_`, and drop out `keep_prob` placeholders using `tf.placeholder`. `labels_` needs to be two-dimensional to work with some functions later.  Since `keep_prob` is a scalar (a 0-dimensional tensor), you shouldn't provide a size to `tf.placeholder`.

In [108]:
n_words = len(vocab_to_int) + 1 # Adding 1 because we use 0's for padding, dictionary started at 1

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(dtype= tf.int32, shape=[None, seq_len], name ='inputs')
    labels_ = tf.placeholder(dtype= tf.int32, shape= [None, 1], name = 'labels')
    keep_prob = tf.placeholder(dtype= tf.float32, name= 'keep_prob')

### Embedding

Now we'll add an embedding layer. We need to do this because there are 74000 words in our vocabulary. It is massively inefficient to one-hot encode our classes here. You should remember dealing with this problem from the word2vec lesson. Instead of one-hot encoding, we can have an embedding layer and use that layer as a lookup table. You could train an embedding layer using word2vec, then load it here. But, it's fine to just make a new layer and let the network learn the weights.

> **Exercise:** Create the embedding lookup matrix as a `tf.Variable`. Use that embedding matrix to get the embedded vectors to pass to the LSTM cell with [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup). This function takes the embedding matrix and an input tensor, such as the review vectors. Then, it'll return another tensor with the embedded vectors. So, if the embedding layer has 200 units, the function will return a tensor with size [batch_size, 200].



In [109]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

In [110]:
embed.shape

TensorShape([Dimension(None), Dimension(200), Dimension(300)])

### LSTM cell

<img src="assets/network_diagram.png" width=400px>

Next, we'll create our LSTM cells to use in the recurrent network ([TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn)). Here we are just defining what the cells look like. This isn't actually building the graph, just defining the type of cells we want in our graph.

To create a basic LSTM cell for the graph, you'll want to use `tf.contrib.rnn.BasicLSTMCell`. Looking at the function documentation:

```
tf.contrib.rnn.BasicLSTMCell(num_units, forget_bias=1.0, input_size=None, state_is_tuple=True, activation=<function tanh at 0x109f1ef28>)
```

you can see it takes a parameter called `num_units`, the number of units in the cell, called `lstm_size` in this code. So then, you can write something like 

```
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

to create an LSTM cell with `num_units`. Next, you can add dropout to the cell with `tf.contrib.rnn.DropoutWrapper`. This just wraps the cell in another cell, but with dropout added to the inputs and/or outputs. It's a really convenient way to make your network better with almost no effort! So you'd do something like

```
drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
```

Most of the time, your network will have better performance with more layers. That's sort of the magic of deep learning, adding more layers allows the network to learn really complex relationships. Again, there is a simple way to create multiple layers of LSTM cells with `tf.contrib.rnn.MultiRNNCell`:

```
cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
```

Here, `[drop] * lstm_layers` creates a list of cells (`drop`) that is `lstm_layers` long. The `MultiRNNCell` wrapper builds this into multiple layers of RNN cells, one for each cell in the list.

So the final cell you're using in the network is actually multiple (or just one) LSTM cells with dropout. But it all works the same from an achitectural viewpoint, just a more complicated graph in the cell.

> **Exercise:** Below, use `tf.contrib.rnn.BasicLSTMCell` to create an LSTM cell. Then, add drop out to it with `tf.contrib.rnn.DropoutWrapper`. Finally, create multiple LSTM layers with `tf.contrib.rnn.MultiRNNCell`.

Here is [a tutorial on building RNNs](https://www.tensorflow.org/tutorials/recurrent) that will help you out.


In [111]:
with graph.as_default():
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

In [112]:
initial_state

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros:0' shape=(500, 256) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(500, 256) dtype=float32>),)

### RNN forward pass

<img src="assets/network_diagram.png" width=400px>

Now we need to actually run the data through the RNN nodes. You can use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn) to do this. You'd pass in the RNN cell you created (our multiple layered LSTM `cell` for instance), and the inputs to the network.

```
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state)
```

Above I created an initial state, `initial_state`, to pass to the RNN. This is the cell state that is passed between the hidden layers in successive time steps. `tf.nn.dynamic_rnn` takes care of most of the work for us. We pass in our cell and the input to the cell, then it does the unrolling and everything else for us. It returns outputs for each time step and the final_state of the hidden layer.

> **Exercise:** Use `tf.nn.dynamic_rnn` to add the forward pass through the RNN. Remember that we're actually passing in vectors from the embedding layer, `embed`.



In [113]:
initial_state

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros:0' shape=(500, 256) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(500, 256) dtype=float32>),)

In [114]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed,
                                             initial_state=initial_state)

In [115]:
outputs.shape

TensorShape([Dimension(500), Dimension(200), Dimension(256)])

In [116]:
type(final_state)

tuple

In [117]:
len(final_state)

1

In [118]:
final_state

(LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(500, 256) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(500, 256) dtype=float32>),)

### Output

We only care about the final output, we'll be using that as our sentiment prediction. So we need to grab the last output with `outputs[:, -1]`, the calculate the cost from that and `labels_`.

In [119]:
outputs[:, -1].shape

TensorShape([Dimension(500), Dimension(256)])

In [120]:
labels_.shape

TensorShape([Dimension(None), Dimension(1)])

> It seems that ouput layer( fully_connected) works untill hidden layer(lstm) all step are finished

In [121]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [122]:
predictions.shape

TensorShape([Dimension(500), Dimension(1)])

### Validation accuracy

Here we can add a few nodes to calculate the accuracy which we'll use in the validation pass.

In [128]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### Batching

This is a simple function for returning batches from our data. First it removes data such that we only have full batches. Then it iterates through the `x` and `y` arrays and returns slices out of those arrays with size `[batch_size]`.

In [123]:
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

## Training

Below is the typical training code. If you want to do this yourself, feel free to delete all this code and implement it yourself. Before you run this, make sure the `checkpoints` directory exists.

In [126]:
epochs = 10

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state) # reset initial_ state in LSTM cell  every epoch
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            print(x.shape)
            feed = {inputs_: x,
                    labels_: y,
                    keep_prob: 0.5,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

(500, 200)


KeyboardInterrupt: 

## Testing

In [ ]:
test_acc = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))